In [1]:
!pip install xgboost

In [2]:
!pip install optuna

     |████████████████████████████████| 308 kB 4.0 MB/s 
     |████████████████████████████████| 81 kB 11.7 MB/s 
     |████████████████████████████████| 210 kB 70.3 MB/s 
     |████████████████████████████████| 75 kB 5.7 MB/s 
     |████████████████████████████████| 113 kB 83.2 MB/s 
     |████████████████████████████████| 49 kB 7.5 MB/s 
     |████████████████████████████████| 150 kB 69.6 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=d3d8ee5a5c70f4fd2460d9eb4694e238a4675e00c6e38ee39fda7bef8a63ed5b
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [3]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

In [4]:
cd ./drive/MyDrive/미니_프로젝트/data

/content/drive/MyDrive/미니_프로젝트/data


# 모든 feature가 다 포함

In [6]:
train_fast = pd.read_csv('./train_fast_final.csv')
test_fast = pd.read_csv('./test_fast_final.csv')
train_slow = pd.read_csv('./train_slow_final.csv')
test_slow = pd.read_csv('./test_slow_final.csv')

In [7]:
train_fast.shape, test_fast.shape, train_slow.shape, test_slow.shape

((102316, 68), (25575, 68), (102294, 68), (25575, 68))

## 급속 충전기

In [8]:
X_fast_train = train_fast.drop(['fast_exist'], axis=1)
y_fast_train = train_fast.fast_exist

X_fast_test = test_fast.drop(['fast_exist'], axis=1)
y_fast_test = test_fast.fast_exist

### XGBoost

In [ ]:
# XGB = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=1, gamma=0,
#               importance_type='gain', interaction_constraints='',
#               learning_rate=0.300000012, max_delta_step=0, max_depth=6,
#               min_child_weight=1,
#               n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=0,
#               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
#               tree_method='exact', validate_parameters=1)

In [ ]:
XGB.fit(X_fast_train, y_fast_train)

XGBClassifier(interaction_constraints='', learning_rate=0.300000012,
              max_depth=6, n_jobs=-1, num_parallel_tree=1, tree_method='exact',
              validate_parameters=1)

In [ ]:
def XGB_objective(trial):
    base_score = 0.5
    booster = trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart'])
    colsample_bylevel = trial.suggest_float('colsample_bylevel', 0, 1)
    colsample_bynode = trial.suggest_float('colsample_bynode', 0, 1)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0, 1)
    gamma = trial.suggest_float('gamma', 0, 10)
    importance_type = trial.suggest_categorical('importance_type', ['weight', 'gain', 'cover', 'total_gain', 'total_cover'])
    learning_rate = trial.suggest_float('learning_rate', 0, 1)
    max_delta_step = trial.suggest_float('max_delta_step', 0, 10)
    max_depth = trial.suggest_int('max_depth', 1, 10)
    min_child_weight = trial.suggest_float('min_child_weight', 0, 10)
    n_estimators =  trial.suggest_int('n_estimators', 100, 500)
    n_jobs = -1
    num_parallel_tree = trial.suggest_int('num_parallel_tree', 1,10)
    random_state = 42
    reg_alpha=0
    reg_lambda=1
    scale_pos_weight=1
    subsample=1
    tree_method='gpu_hist'
    validate_parameters=1
    
   
    model = XGBClassifier(base_score=base_score, booster=booster, colsample_bylevel=colsample_bylevel, colsample_bytree=colsample_bytree, colsample_bynode=colsample_bynode,
                          gamma=gamma, importance_type=importance_type, learning_rate=learning_rate, max_delta_step=max_delta_step, max_depth=max_depth, min_child_weight=min_child_weight,
                          n_estimators=n_estimators, n_jobs=n_jobs, num_parallel_tree=num_parallel_tree, random_state=random_state, reg_alpha=reg_alpha, reg_lambda=reg_lambda,
                          scale_pos_weight=scale_pos_weight, subsample=subsample, tree_method=tree_method, validate_parameters=validate_parameters
                          )

    model.fit(X_fast_train, y_fast_train)
    y_fast_pred = model.predict(X_fast_test)
    f1 = f1_score(y_fast_test, y_fast_pred)

    return f1
    
#Execute optuna and set hyperparameters
XGB_study = optuna.create_study(direction='maximize')
XGB_study.optimize(XGB_objective, n_trials=100)


#Create an instance with tuned hyperparameters
optimized_XGB = XGBClassifier(base_score=0.5, booster=XGB_study.best_params['booster'], colsample_bylevel=XGB_study.best_params['colsample_bylevel'], colsample_bytree=XGB_study.best_params['colsample_bytree'], 
                              colsample_bynode=XGB_study.best_params['colsample_bynode'], gamma=XGB_study.best_params['gamma'], importance_type=XGB_study.best_params['importance_type'], learning_rate=XGB_study.best_params['learning_rate'], 
                              max_delta_step=XGB_study.best_params['max_delta_step'], max_depth=XGB_study.best_params['max_depth'], min_child_weight=XGB_study.best_params['min_child_weight'], n_estimators=XGB_study.best_params['n_estimators'], 
                              n_jobs=-1, num_parallel_tree=XGB_study.best_params['num_parallel_tree'], random_state=42, reg_alpha=0, 
                              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='gpu_hist', 
                              validate_parameters=1
                              )

[I 2022-02-23 06:32:47,142] A new study created in memory with name: no-name-d8cacc95-7034-4f0d-a23f-904ec544686c
[I 2022-02-23 06:33:03,387] Trial 0 finished with value: 0.03290414878397711 and parameters: {'booster': 'gblinear', 'colsample_bylevel': 0.8806393294214206, 'colsample_bynode': 0.2686838234067269, 'colsample_bytree': 0.5110414100360772, 'gamma': 2.5276004665153873, 'importance_type': 'gain', 'learning_rate': 0.05420613350824699, 'max_delta_step': 5.053454896326979, 'max_depth': 6, 'min_child_weight': 6.233565123573791, 'n_estimators': 408, 'num_parallel_tree': 5}. Best is trial 0 with value: 0.03290414878397711.
[I 2022-02-23 06:33:08,837] Trial 1 finished with value: 0.03290414878397711 and parameters: {'booster': 'gblinear', 'colsample_bylevel': 0.7420392359292561, 'colsample_bynode': 0.3214867279184648, 'colsample_bytree': 0.2704669509016083, 'gamma': 7.25589134262631, 'importance_type': 'total_gain', 'learning_rate': 0.13304361158033184, 'max_delta_step': 4.07944430388

KeyboardInterrupt: ignored

[I 2022-02-23 06:37:42,546] Trial 4 finished with value: 0.7450980392156863 and parameters: {'booster': 'dart', 'colsample_bylevel': 0.543720424886226, 'colsample_bynode': 0.9264769381191881, 'colsample_bytree': 0.5726686892518192, 'gamma': 7.819103771557197, 'importance_type': 'weight', 'learning_rate': 0.5638340291833598, 'max_delta_step': 2.720148350536493, 'max_depth': 4, 'min_child_weight': 1.6655844065138026, 'n_estimators': 381, 'num_parallel_tree': 3}. Best is trial 4 with value: 0.7450980392156863.


In [ ]:
optuna.visualization.plot_param_importances(XGB_study)

In [ ]:
optuna.visualization.plot_optimization_history(XGB_study)

In [ ]:
param = {'booster': 'dart', 'colsample_bylevel': 0.543720424886226, 'colsample_bynode': 0.9264769381191881, 'colsample_bytree': 0.5726686892518192, 
         'gamma': 7.819103771557197, 'importance_type': 'weight', 'learning_rate': 0.5638340291833598, 'max_delta_step': 2.720148350536493, 'max_depth': 4, 
         'min_child_weight': 1.6655844065138026, 'n_estimators': 381, 'num_parallel_tree': 3, 'base_score': 0.5, 'n_jobs': -1, 'random_state': 42,
         'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 1, 'validate_parameters': 1
         }

In [ ]:
optimized_XGB = XGBClassifier(**param)

In [ ]:
optimized_XGB.fit(X_fast_train, y_fast_train)

XGBClassifier(booster='dart', colsample_bylevel=0.543720424886226,
              colsample_bynode=0.9264769381191881,
              colsample_bytree=0.5726686892518192, gamma=7.819103771557197,
              learning_rate=0.5638340291833598,
              max_delta_step=2.720148350536493, max_depth=4,
              min_child_weight=1.6655844065138026, n_estimators=381, n_jobs=-1,
              num_parallel_tree=3, random_state=42, validate_parameters=1)

In [ ]:
y_fast_prob = optimized_XGB.predict_proba(X_fast_test)

In [ ]:
y_fast_pred = optimized_XGB.predict(X_fast_test)
f1_score(y_fast_test, y_fast_pred)

0.5588235294117647

In [ ]:
for threshold in range(1,10):
    threshold = round(threshold*0.1, 1)
    y_fast_pred = np.where(y_fast_prob>=threshold, 1, 0)
    print(f'{threshold} :{f1_score(y_fast_test, y_fast_pred[:,1])}')

0.1 :0.3529411764705882
0.2 :0.4375
0.3 :0.4367816091954023
0.4 :0.5066666666666666
0.5 :0.5588235294117647
0.6 :0.59375
0.7 :0.6037735849056605
0.8 :0.6274509803921569
0.9 :0.5957446808510639


In [ ]:
XGB = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1,
              n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1)

In [11]:
XGB.fit(X_fast_train, y_fast_train)

XGBClassifier(interaction_constraints='', learning_rate=0.300000012,
              max_depth=6, n_jobs=-1, num_parallel_tree=1, tree_method='exact',
              validate_parameters=1)

In [12]:
y_fast_prob = XGB.predict_proba(X_fast_test)

In [13]:
for threshold in range(1,10):
    threshold = round(threshold*0.1, 1)
    y_fast_pred = np.where(y_fast_prob>=threshold, 1, 0)
    print(f'{threshold} :{f1_score(y_fast_test, y_fast_pred[:,1])}')

0.1 :0.6666666666666667
0.2 :0.7241379310344828
0.3 :0.7272727272727272
0.4 :0.7407407407407408
0.5 :0.7058823529411765
0.6 :0.68
0.7 :0.6530612244897959
0.8 :0.6521739130434783
0.9 :0.6363636363636364


* **오버피팅!!!!!!!!!**

In [14]:
y_fast_pred_train = XGB.predict(X_fast_train)
f1_score(y_fast_train, y_fast_pred_train)

1.0

## 완속 충전기

In [15]:
X_slow_train = train_slow.drop(['slow_exist'], axis=1)
y_slow_train = train_slow.slow_exist

X_slow_test = test_slow.drop(['slow_exist'], axis=1)
y_slow_test = test_slow.slow_exist

### XGBoost

In [20]:
XGB_slow = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1,
              n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1)

In [21]:
XGB_slow.fit(X_slow_train, y_slow_train)

XGBClassifier(interaction_constraints='', learning_rate=0.300000012,
              max_depth=6, n_jobs=-1, num_parallel_tree=1, tree_method='exact',
              validate_parameters=1)

In [22]:
y_slow_prob = XGB_slow.predict_proba(X_slow_test)

In [23]:
for threshold in range(1,10):
    threshold = round(threshold*0.1, 1)
    y_slow_pred = np.where(y_slow_prob>=threshold, 1, 0)
    print(f'{threshold} :{f1_score(y_slow_test, y_slow_pred[:,1])}')

0.1 :0.0
0.2 :0.0
0.3 :0.0
0.4 :0.0
0.5 :0.0
0.6 :0.0
0.7 :0.0
0.8 :0.0
0.9 :0.0
